# Part 4: Semantic Kernel Agent Framework

This notebook demonstrates Semantic Kernel's Agent Framework for multi-agent orchestration. We'll explore agent groups, chat completion agents, and structured enterprise patterns that showcase SK's advanced capabilities.

## Environment Setup

Setting up the environment for Semantic Kernel Agent Framework with Azure OpenAI.

In [ ]:
# Environment and configuration setup for SK Agent Framework
import os
import warnings
import asyncio
from pathlib import Path
from dotenv import load_dotenv, find_dotenv
from typing import List, Dict, Any, Optional

# Import Semantic Kernel components
import semantic_kernel as sk
from semantic_kernel.agents import ChatCompletionAgent
from semantic_kernel.agents.group_chat import AgentGroupChat
from semantic_kernel.agents.strategies import TerminationStrategy
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion, AzureChatPromptExecutionSettings
from semantic_kernel.contents import ChatHistory, ChatMessageContent
from semantic_kernel.connectors.ai import FunctionChoiceBehavior

class SKAgentConfig:
    """Configuration management for SK Agent Framework"""
    
    def __init__(self):
        # Load environment variables
        env_path = find_dotenv()
        if env_path:
            load_dotenv(env_path)
            print(f"✅ Loaded environment from: {env_path}")
        else:
            warnings.warn("No .env file found. Using system environment variables only.")
        
        self._load_azure_config()
        self._validate_config()
    
    def _load_azure_config(self):
        """Load Azure OpenAI configuration from environment variables"""
        self.azure_api_key = os.getenv('AZURE_OPENAI_API_KEY')
        self.azure_endpoint = os.getenv('AZURE_OPENAI_ENDPOINT')
        self.azure_deployment = os.getenv('AZURE_OPENAI_CHAT_DEPLOYMENT_NAME', 'gpt-4o-mini')
        self.azure_api_version = os.getenv('AZURE_OPENAI_API_VERSION', '2024-10-21')
        self.service_id = "azure_chat"
    
    def _validate_config(self):
        """Validate critical configuration"""
        errors = []
        
        if not self.azure_api_key:
            errors.append("AZURE_OPENAI_API_KEY is required")
        
        if not self.azure_endpoint:
            errors.append("AZURE_OPENAI_ENDPOINT is required")
        
        if errors:
            raise ValueError(f"Configuration errors: {', '.join(errors)}")
        
        print("✅ All SK Agent Framework configuration validated successfully")
    
    def create_kernel(self) -> sk.Kernel:
        """Create a configured kernel instance"""
        kernel = sk.Kernel()
        
        # Add Azure OpenAI chat completion service
        azure_chat_service = AzureChatCompletion(
            service_id=self.service_id,
            deployment_name=self.azure_deployment,
            endpoint=self.azure_endpoint,
            api_key=self.azure_api_key,
            api_version=self.azure_api_version
        )
        
        kernel.add_service(azure_chat_service)
        return kernel

# Initialize configuration
config = SKAgentConfig()
kernel = config.create_kernel()
print(f"🤖 SK Agent Framework Configuration Loaded")
print(f"   Services in kernel: {len(kernel.services)}")

## 4.1 Creating Specialized Chat Completion Agents

SK's Agent Framework allows creating specialized agents with distinct roles, instructions, and capabilities.

In [ ]:
# Create specialized plugins for our agents
class ResearchPlugin:
    """Plugin providing research capabilities"""
    
    @sk.kernel_function(
        description="Research a given topic and return findings",
        name="research_topic"
    )
    def research_topic(self, topic: str) -> str:
        """Research a topic and return findings"""
        research_database = {
            "artificial intelligence": "AI technologies are advancing rapidly with developments in large language models, computer vision, robotics, and edge AI deployment. Key trends include multimodal AI systems, ethical AI frameworks, and enterprise AI adoption.",
            "climate change": "Climate change remains a critical global challenge with accelerating impacts including rising sea levels, extreme weather events, and ecosystem disruption. Mitigation strategies focus on renewable energy, carbon capture, and sustainable practices.",
            "quantum computing": "Quantum computing is approaching practical applications with companies like IBM, Google, and Rigetti achieving quantum advantage in specific domains. Focus areas include error correction, quantum algorithms, and hybrid classical-quantum systems.",
            "blockchain": "Blockchain technology continues evolving beyond cryptocurrencies into enterprise applications including supply chain management, digital identity, smart contracts, and decentralized finance (DeFi) platforms."
        }
        
        for key, data in research_database.items():
            if key.lower() in topic.lower():
                return f"Research findings on {key}: {data}"
        
        return f"Limited research data available for topic: {topic}"

class AnalysisPlugin:
    """Plugin providing analysis capabilities"""
    
    @sk.kernel_function(
        description="Analyze research data and provide insights",
        name="analyze_data"
    )
    def analyze_data(self, data: str, focus: str = "trends") -> str:
        """Analyze research data with specific focus"""
        if "AI" in data or "artificial intelligence" in data.lower():
            return "Analysis: AI sector demonstrates exponential growth trajectory with significant enterprise adoption. Investment patterns indicate focus on practical applications and ethical frameworks. Recommendation: Monitor regulatory developments and enterprise integration strategies."
        elif "climate" in data.lower():
            return "Analysis: Climate data reveals urgent acceleration in environmental changes requiring immediate action. Economic analysis shows growing investment in clean technology and carbon markets. Recommendation: Prioritize adaptation strategies alongside mitigation efforts."
        elif "quantum" in data.lower():
            return "Analysis: Quantum computing approaching commercial viability with potential disruption in cryptography, optimization, and simulation. Market analysis indicates substantial corporate and government investment. Recommendation: Prepare for quantum-safe security measures."
        elif "blockchain" in data.lower():
            return "Analysis: Blockchain adoption moving beyond speculation toward practical enterprise applications. Analysis shows maturation in regulatory frameworks and technical infrastructure. Recommendation: Focus on interoperability and scalability solutions."
        else:
            return f"General analysis: The provided data suggests emerging trends requiring further investigation and strategic consideration."

# Add plugins to kernel
kernel.add_plugin(ResearchPlugin(), plugin_name="Research")
kernel.add_plugin(AnalysisPlugin(), plugin_name="Analysis")

print("🔌 Plugins registered for Agent Framework:")
for plugin_name, plugin in kernel.plugins.items():
    function_names = list(plugin.functions.keys())
    print(f"  • {plugin_name}: {function_names}")

## 4.2 Chat Completion Agent Creation

Let's create specialized agents with different roles and capabilities using the Agent Framework.

In [ ]:
# Create specialized Chat Completion Agents

# Research Agent
research_agent = ChatCompletionAgent(
    service_id=config.service_id,
    kernel=kernel,
    name="ResearchAgent",
    instructions="""
    You are a thorough research specialist focused on gathering accurate, comprehensive information.
    
    Your responsibilities:
    - Use the research_topic function to gather relevant data
    - Provide factual, well-sourced information
    - Identify key trends and developments
    - Maintain objectivity and accuracy
    
    Always use the available research tools and provide detailed findings.
    """,
    execution_settings=AzureChatPromptExecutionSettings(
        service_id=config.service_id,
        max_tokens=300,
        temperature=0.3,  # Lower temperature for factual accuracy
        function_choice_behavior=FunctionChoiceBehavior.AUTO
    )
)

# Analysis Agent
analysis_agent = ChatCompletionAgent(
    service_id=config.service_id,
    kernel=kernel,
    name="AnalysisAgent",
    instructions="""
    You are an expert analyst specializing in interpreting research data and providing strategic insights.
    
    Your responsibilities:
    - Use the analyze_data function to process research findings
    - Identify patterns, trends, and implications
    - Provide actionable recommendations
    - Consider economic, technical, and strategic factors
    
    Focus on delivering clear, actionable insights from complex data.
    """,
    execution_settings=AzureChatPromptExecutionSettings(
        service_id=config.service_id,
        max_tokens=300,
        temperature=0.5,  # Medium temperature for balanced analysis
        function_choice_behavior=FunctionChoiceBehavior.AUTO
    )
)

# Synthesis Agent (Report Writer)
synthesis_agent = ChatCompletionAgent(
    service_id=config.service_id,
    kernel=kernel,
    name="SynthesisAgent",
    instructions="""
    You are a skilled technical writer and synthesizer who creates comprehensive reports.
    
    Your responsibilities:
    - Synthesize research and analysis into coherent reports
    - Create executive summaries and key findings
    - Provide clear recommendations and next steps
    - Ensure professional formatting and structure
    
    Create well-structured, actionable reports that decision-makers can use effectively.
    """,
    execution_settings=AzureChatPromptExecutionSettings(
        service_id=config.service_id,
        max_tokens=400,
        temperature=0.7,  # Higher temperature for creative synthesis
        function_choice_behavior=FunctionChoiceBehavior.NONE  # No tools needed for synthesis
    )
)

print("🤖 Chat Completion Agents Created:")
print(f"   🔍 ResearchAgent: Specialized in data gathering (temp=0.3)")
print(f"   📈 AnalysisAgent: Focused on insights and patterns (temp=0.5)")
print(f"   ✍️ SynthesisAgent: Expert in report creation (temp=0.7)")
print(f"   🔧 All agents configured with Azure OpenAI and appropriate function calling")

## 4.3 Agent Group Chat Implementation

The Agent Framework provides structured group chat capabilities for coordinated multi-agent workflows.

In [ ]:
# Create Agent Group Chat for coordinated workflow
from semantic_kernel.agents.strategies import SequentialSelectionStrategy

# Define custom termination strategy
class CompletionTerminationStrategy(TerminationStrategy):
    """Custom termination strategy for our research workflow"""
    
    def __init__(self, max_iterations: int = 10):
        super().__init__()
        self.max_iterations = max_iterations
        self.iteration_count = 0
    
    async def should_terminate(self, history: List[ChatMessageContent]) -> bool:
        """Determine if the conversation should terminate"""
        self.iteration_count += 1
        
        # Terminate if max iterations reached
        if self.iteration_count >= self.max_iterations:
            return True
        
        # Check if we have a complete workflow (research -> analysis -> synthesis)
        if len(history) >= 3:
            recent_messages = [msg.content for msg in history[-3:]]
            
            # Look for completion indicators
            synthesis_keywords = ["executive summary", "recommendations", "final report", "conclusion"]
            last_message = recent_messages[-1].lower() if recent_messages else ""
            
            if any(keyword in last_message for keyword in synthesis_keywords):
                return True
        
        return False

# Create the agent group chat with sequential selection
agent_group_chat = AgentGroupChat(
    agents=[research_agent, analysis_agent, synthesis_agent],
    selection_strategy=SequentialSelectionStrategy(),
    termination_strategy=CompletionTerminationStrategy(max_iterations=6)
)

print("👥 Agent Group Chat Configured:")
print(f"   🔄 Strategy: Sequential Selection (Research → Analysis → Synthesis)")
print(f"   ⏹️ Termination: Custom completion detection with max 6 iterations")
print(f"   🤖 Agents: {len(agent_group_chat.agents)} specialized agents")

## 4.4 Executing Multi-Agent Workflows

Let's execute our Agent Framework workflow and see how the agents collaborate through structured group chat.

In [ ]:
async def run_agent_framework_workflow(topic: str):
    """Execute the multi-agent workflow using SK Agent Framework"""
    
    print(f"🚀 Starting SK Agent Framework Workflow")
    print(f"📝 Research Topic: {topic}")
    print("=" * 60)
    
    # Create initial message
    initial_message = f"""
    Please conduct a comprehensive research and analysis on the topic: {topic}
    
    The workflow should include:
    1. Research Agent: Gather current information and data
    2. Analysis Agent: Analyze the research and identify key insights
    3. Synthesis Agent: Create a comprehensive report with recommendations
    
    Please begin with the research phase.
    """
    
    try:
        # Initialize the chat history
        chat_history = ChatHistory()
        chat_history.add_user_message(initial_message)
        
        print("🤖 Agents beginning collaboration...\n")
        
        # Execute the agent group chat
        async for response in agent_group_chat.invoke_stream(chat_history):
            agent_name = response.author.name if hasattr(response, 'author') else "Unknown"
            
            print(f"🎯 {agent_name}:")
            print(f"   {response.content[:200]}{'...' if len(response.content) > 200 else ''}")
            print(f"   📊 Full response length: {len(response.content)} characters")
            print("-" * 50)
        
        print("\n" + "=" * 60)
        print("✅ AGENT FRAMEWORK WORKFLOW COMPLETED")
        print("=" * 60)
        
        # Display final conversation summary
        final_history = agent_group_chat.history
        print(f"\n📊 Workflow Summary:")
        print(f"   Total messages: {len(final_history)}")
        print(f"   Agents participated: {len(set(msg.author.name for msg in final_history if hasattr(msg, 'author')))}")
        
        # Show the final synthesis
        if final_history:
            final_message = final_history[-1]
            print(f"\n📝 Final Report Preview:")
            print(f"   {final_message.content[:400]}...")
        
        return final_history
        
    except Exception as e:
        print(f"❌ Workflow error: {e}")
        return None

# Test the Agent Framework with different topics
test_topics = [
    "artificial intelligence in healthcare",
    "quantum computing applications",
    "blockchain in supply chain management"
]

for topic in test_topics:
    print(f"\n{'='*70}")
    result = await run_agent_framework_workflow(topic)
    if result:
        print(f"✅ Successfully completed Agent Framework workflow for: {topic}")
    else:
        print(f"❌ Failed to complete workflow for: {topic}")
    print(f"{'='*70}")
    
    # Add a brief pause between workflows
    await asyncio.sleep(1)

## Summary: Semantic Kernel Agent Framework

This notebook demonstrated Semantic Kernel's enterprise-ready Agent Framework:

### Key Concepts Learned:
1. **Chat Completion Agents**: Specialized agents with distinct roles, instructions, and capabilities
2. **Agent Group Chat**: Structured multi-agent coordination with selection strategies
3. **Function Choice Behavior**: Controlled tool usage per agent role
4. **Termination Strategies**: Custom logic for workflow completion

### SK Agent Framework Advantages:
- **Enterprise Architecture**: Built-in governance, logging, and control mechanisms
- **Structured Coordination**: Selection strategies and termination conditions
- **Role Specialization**: Different instructions, temperatures, and tool access per agent
- **Azure Integration**: Seamless integration with Microsoft ecosystem

### Agent Framework vs LangGraph:
| Aspect | LangGraph | SK Agent Framework |
|--------|-----------|-------------------|
| **Architecture** | State graphs with custom functions | Chat completion agents with instructions |
| **Coordination** | Graph-based flow control | Selection strategies and group chat |
| **Enterprise Focus** | Community-driven patterns | Built-in enterprise governance |
| **Tool Management** | Dynamic tool binding | Structured function choice behavior |
| **State Management** | Custom TypedDict states | Chat history and agent context |

### Production Benefits:
- **Governance**: Built-in agent management and oversight capabilities
- **Scalability**: Easy to add new agents and modify selection strategies
- **Monitoring**: Rich logging and telemetry integration
- **Security**: Enterprise-grade authentication and authorization patterns

### Next Steps:
- **Advanced Selection Strategies**: Custom routing logic based on conversation context
- **Agent Memory**: Persistent memory across conversation sessions
- **Integration Patterns**: Connecting with Azure services and enterprise systems
- **Monitoring & Observability**: Production monitoring and performance optimization

### Production Considerations:
- Implement comprehensive error handling and fallback strategies
- Design selection strategies that optimize for both performance and cost
- Use Azure Monitor integration for production observability
- Consider agent resource allocation and scaling patterns
- Implement proper security boundaries for enterprise deployment